In [1]:
def read_cif_file(file, flag):
    #specify full relative path, flag = true
        # blank list to store the cif data
    cif_data = []
        # blank list to store the entire cif file
    cif_file = []
        # the flag variable has been introduced as a marker to activate when the cif data is to be read and switched off
        # when it doesn't have to be read.
    flag1 = 1
    flag2 = 1
    flag3 = 1
    if flag == False:
        keywords = "_atom_site_fract_z"
    if flag == True:
        keywords = "_atom_site_fract_z"
    exit_keyword = "#END"
    exit_keyword_2 = "loop_"
        # get the current working directory
    cwd = os.getcwd()
    if flag == False:
        path = "%s/%s" %(cwd, file)
    if flag == True:
        path = "%s" % file
        # in this case the file is opened using this syntax as the syntax used to open a file in the output_parser
        # function was not working for this case
    f = open(path, "r")
    for x in f:
        if (exit_keyword in x) or (exit_keyword_2 in x):
            flag1 = 1
        if flag1 == 2:
            x = x.strip()
            x = x.split()
                # the if statements below along with the for loops are used to remove the brackets (and the data contained within these brackets)
            for i in range(0, len(x)):
                x[i] = remove_brackets_from_line(x[i])
            cif_data.append(x)
                # Creating a copy of list y, so I can append "\n" to the cif data to make it easier to print
            y = x.copy()
            y.append("\n")
            cif_file.append(y)
        if flag1 == 1:
            cif_file.append(x)
        if keywords in x:
            flag1 = 2
    return (cif_data, cif_file)
def remove_brackets_from_line(line):
    flag3 = 1
    for i in range(0, len(line)):
        if line[i] == ')':
            flag2 = 1
            stop = i
            flag3 = 2
        if line[i] == '(':
            flag2 = 2
            start = i
    if flag3 == 2:
        line = line[0: start:] + line[stop + 1::]
    return (line)
#[x,y] = read_cif_file(tag, True)

[x,y] = read_cif_file('Movies/System_0/Framework_0_initial_2_2_2_P1.cif', True)



In [14]:
import numpy as np
def calc_number_of_unit_cells(unit_cell_dimensions):
    #remember to check the cutoff everytime you run the program
    cutoff = 12.9
    length_a = unit_cell_dimensions[0]
    length_b = unit_cell_dimensions[1]
    length_c = unit_cell_dimensions[2]
    alpha = unit_cell_dimensions[3]
    beta = unit_cell_dimensions[4]
    gamma = unit_cell_dimensions[5]
    # Convert cif information to unit_cell vectors
    ax = length_a
    ay = 0.0
    az = 0.0
    bx = length_b * np.cos(gamma * np.pi / 180.0)
    by = length_b * np.sin(gamma * np.pi / 180.0)
    bz = 0.0
    cx = length_c * np.cos(beta * np.pi / 180.0)
    cy = (length_c * length_b * np.cos(alpha * np.pi / 180.0) - bx * cx) / by
    cz = (length_c ** 2 - cx ** 2 - cy ** 2) ** 0.5
    unit_cell = np.asarray([[ax, ay, az], [bx, by, bz], [cx, cy, cz]])
    # Unit cell vectors
    A = unit_cell[0]
    B = unit_cell[1]
    C = unit_cell[2]
    # minimum distances between unit cell faces
    Wa = np.divide(np.linalg.norm(np.dot(np.cross(B, C), A)), np.linalg.norm(np.cross(B, C)))
    Wb = np.divide(np.linalg.norm(np.dot(np.cross(C, A), B)), np.linalg.norm(np.cross(C, A)))
    Wc = np.divide(np.linalg.norm(np.dot(np.cross(A, B), C)), np.linalg.norm(np.cross(A, B)))
    uc_x = int(np.ceil(cutoff / (0.5 * Wa)))
    uc_y = int(np.ceil(cutoff / (0.5 * Wb)))
    uc_z = int(np.ceil(cutoff / (0.5 * Wc)))
    number_of_unit_cells = [uc_x, uc_y, uc_z]
    return (number_of_unit_cells)



def extract_unit_cell_dimensions(cif_file):
    # List of all unit cell dimensions in the following order: length_a, length_b, length_c, alpha, gamma, beta
    unit_cell_dimensions = []
    for line in cif_file:
        flag3 = 1
        if "_cell_length_a" in line:
            line= line.split()[1]  # unit cell vector
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))  # string to float
        if "_cell_length_b" in line:
            line = line.split()[1]
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))
        if "_cell_length_c" in line:
            line = line.split()[1]
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))
        if "_cell_angle_alpha" in line:
            line = line.split()[1]
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))
        if "_cell_angle_beta" in line:
            line = line.split()[1]
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))
        if "_cell_angle_gamma" in line:
            line = line.split()[1]
            line = remove_brackets_from_line(line)
            unit_cell_dimensions.append(float(line))
    return(unit_cell_dimensions)


unit_cell_dimensions = extract_unit_cell_dimensions(y)
calc_number_of_unit_cells(unit_cell_dimensions)

[1, 1, 1]

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import distance

metal_atoms_set = {'zn'}

with open('RASPA_Output/IRMOF-1.xyz') as file:
    num_atoms = int(file.readline())
    MOF_name = file.readline()
    print(num_atoms)
    print(MOF_name)
        
    atom = []
    atom_ids = []
    points = []
    
    atom_id = 1

    for line in file:
        atom_name, x, y, z = line.split()
        atom_name = atom_name.lower()
        atom.append((atom_name, atom_id))

        # Extract metal atoms to visualise on a 3D scatter plot.
        if atom_name in metal_atoms_set:
            x = float(x)
            y = float(y)
            z = float(z)
            atom_ids.append(atom_id)
            point = (x, y, z)
            points.append(point)
        
        atom_id += 1

x_locs = [p[0] for p in points]
y_locs = [p[1] for p in points]
z_locs = [p[2] for p in points]

points = np.array(points)
centroid = points.mean(axis=0)


dist_array = [distance.euclidean(centroid, point) for point in points]
# print(min(dist_array), np.argmin(dist_array))
# print(atom_ids[197])
# print(atom[2579])


df = pd.read_table(inputfile, skiprows=2, delim_whitespace=True, names=['atom', 'x', 'y', 'z'])
centroid=[np.mean(df['x'].tolist(), axis=0), np.mean(df['y'].tolist(), axis=0), np.mean(df['z'].tolist(), axis=0)]
print(centroid)

3392
IRMOF-1

10.696153427111048 197
2580
('zn', 2580)
